In [1]:
import requests
import urllib
import pandas as pd 
from bs4 import BeautifulSoup
import json
import geopandas as gpd

In [2]:
#Load Daily data
page = requests.get("https://info.gesundheitsministerium.at/data/Trend.js")
page.content
soup = BeautifulSoup(page.content, 'html.parser')
a = [item for item in soup.children]
jsonStr = a[0].split("var dpTrend = ")[1].split(";")[0]
jsonData = json.loads(jsonStr)
df = pd.DataFrame(jsonData)
df.columns = ['date', 'cases']
df["date"] = df["date"]+".2020"
df.to_csv("austria_confirmed.csv",index = False)

In [3]:
page = requests.get("https://info.gesundheitsministerium.at/data/Trend.js")
page.content
soup = BeautifulSoup(page.content, 'html.parser')
a = [item for item in soup.children]
jsonStr = a[0].split("var dpTrend = ")[1].split(";")[0]
jsonData = json.loads(jsonStr)
df = pd.DataFrame(jsonData)
df.columns = ['date', 'cases']
df["date"] = df["date"]+".2020"
df.to_csv("austria_confirmed.csv",index = False)

In [99]:
#Load City Data 
import codecs
page = requests.get("https://info.gesundheitsministerium.at/data/Bezirke.js")
page.content
soup = BeautifulSoup(page.content, 'html.parser')
a = [item for item in soup.children]
jsonStr = a[0].split("var dpBezirke = ")[1].split(";")[0]
jsonData = json.loads(jsonStr)
df = pd.DataFrame(jsonData)
df.columns = ['name', 'cases']
gdf = gpd.read_file('STATISTIK_AUSTRIA_POLBEZ_20200101Polygon.shp',encoding="UTF-8")
soa_shape_map_geo = gdf.to_crs(epsg=4326)
soa_shape_map_geo["x"] = soa_shape_map_geo.geometry.centroid.x
soa_shape_map_geo["y"] = soa_shape_map_geo.geometry.centroid.y
soa_shape_map_geo = soa_shape_map_geo.merge(df,on = "name")
soa_shape_map_geo["id"] = soa_shape_map_geo["id"].astype(int)

In [5]:
death = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
death = death[death["Country/Region"] == "Austria"]
death = death.iloc[:,49:]
import datetime
columns = [datetime.datetime.strptime(i, '%m/%d/%y').strftime("%d.%m.%Y") for i in death.columns]
death.columns = columns
death.to_csv("austria_death.csv",index=False)

In [93]:
einwohner = pd.read_csv("einwohnerzahl_1.1.2019_nach_gemeinden_mit_status_gebietsstand_1.1.csv",
                       delimiter=";",skiprows=1)
einwohner["id"] = einwohner['Gemeinde-kennziffer (GKZ)']//100
einwohner["Bevölkerung am 1.1.2019"] = einwohner["Bevölkerung am 1.1.2019"].replace("\.","",regex = True).astype(int)

In [96]:
einwohner = einwohner.groupby(by="id").sum()

In [102]:
soa_shape_map_geo = soa_shape_map_geo.merge(einwohner,on = "id")

In [103]:
soa_shape_map_geo.to_csv("austria_gdf.csv",index = False)